In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob, os
import csv
import re

### replace fb21* with your genome of interest

In [ ]:
#you need to download or use via ftp some version of the arcog database

cog_def = pd.read_csv("ar14.arCOGdef18.tab", sep='\t', lineterminator='\n', header=None)
cog_def.columns = ['ID', 'function', 'gene', 'product']
cog_def


In [ ]:
cog_prod = {}
cog_name = {}

for ind, row in cog_def.iterrows():
    cog_prod[row['ID']] = row['product']
    cog_name[row['ID']] = row['gene']
    


In [ ]:
#load in gff file, which is the genome you want to annotate. 


with open('fb21.def.gff', 'r') as gff:
    for line in gff:
        line = line.strip()
        if not line:
            continue
        if 'arCOG' in line and 'hypothetical protein' in line:
            fam_num = 'arCOG' + (line.split('arCOG')[2][:5])
            #print(line[:-20])
            print(line[:-20] + cog_prod[fam_num],file=open('fb21.def.renamed.gff', 'a'))
        

In [ ]:
gff_def = pd.read_csv("gff/fb21.gff", sep='\t', lineterminator='\n', header=None)
gff_def.columns = ['contig', 'method', 'type', 'start', 'stop', 'mark', 'strand', 'zero', 'tags']
gff_def

        

In [ ]:
name_switch = {}
accession_switcheroo = {}
for ind, row in gff_def.iterrows():
    head = row['tags'].split('product=',1)[1]
    current = (row['contig'] + ':' + str(row['start'] -1) + '-' + str(row['stop']) + '(' + row['strand'] + ')')
    name_switch[current] = head
    accession_switcheroo[current] = row['tags'][3:17]
accession_switcheroo

In [ ]:
#for the ffn or nucleotide sequences: locus tag | position | product
with open('fb21.ffn', 'r') as ffn_bad:
    for line in ffn_bad:
        line = line.strip()
        if not line:
            continue
        if line.startswith('>'):
            print('>' + accession_switcheroo[line[1:]] + '|' + line[1:] + '|' +name_switch[line[1:]],file=open('fb21.renamed.ffn', 'a'))
        if not line.startswith('>'):
            print(line, file=open('fb21.renamed.ffn', 'a'))


In [ ]:
rev_accession_switcheroo = {y:x for x,y in accession_switcheroo.items()}
rev_accession_switcheroo

In [ ]:
#you can do this for amino acid seqs too
with open('hez_fb21.faa', 'r') as faa:
    for line in faa:
        line = line.strip()
        if not line:
            continue
        if line.startswith('>'):
            loci = line[1:15]
            if loci in rev_accession_switcheroo.keys():
                print('>' + loci + '|' + rev_accession_switcheroo[loci] + "|" + name_switch[rev_accession_switcheroo[loci]], file=open('fb21.renamed.faa','a'))
            else:
                print('>' + line[1:], file=open('fb21.renamed.faa','a'))
        if not line.startswith('>'):
            print(line, file=open('fb21.renamed.faa','a'))